In [1]:
# import packages we need to use
import scipy.stats
import math
import pandas as pd
from matplotlib import pyplot as plt
from ipywidgets import widgets
import plotly.graph_objects as go

In [2]:
# Preparing data

# Confirmed cases
link1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv';
# Deaths
link2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv';

#dropping unnecessary variables and aggregating by state:
def trimedData(link):
    df =  pd.read_csv(link,sep=',',header=0)
    df = df[df.iso3 =='USA']
    df = df[df.Lat > 0.0]; #getting rid of some "unassigned" locations
    df = df.drop(['UID', 'FIPS', 'Admin2','iso2', 'iso3', 'code3','Lat', 'Long_', 'Combined_Key', 'Country_Region'], axis=1);
    return df.groupby(['Province_State']).sum()

cnfStates = trimedData(link1);
dthStates = trimedData(link2);

In [3]:
# new class definition

class State: 
    def __init__(self, name, buffer, lag):
        self.name = name
        self.buffer = buffer
        self.lag = lag
        self.confirmed = []
        self.deaths = []
        self.dates = []
        self.nobs = 0
    
    def create(self):        
        first = self.buffer
        last = cnfStates.shape[1]
        self.nobs = last - first 
        self.confirmed = (cnfStates.loc[self.name][first:last].to_numpy()
                          - cnfStates.loc[self.name][first-self.lag:last-self.lag].to_numpy())/self.lag
        self.deaths = (dthStates.loc[self.name][first:last].to_numpy() 
                       - dthStates.loc[self.name][first-self.lag:last-self.lag].to_numpy())/self.lag
        self.dates = cnfStates.columns[first:last]
        
    def show(self):
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=self.dates,      
            y=self.confirmed,
            name="Confirmed"
        ))

        fig.add_trace(go.Scatter(
    
            x=self.dates,    
            y=self.deaths,
            name="Deaths",
            yaxis="y2"
        ))
        
        fig.update_layout(
            xaxis=dict(
                domain=[0.06, 0.94]
            ),
            yaxis=dict(
                title="Confirmed",
                titlefont=dict(
                    color="#1f77b4"
                ),
                tickfont=dict(
                    color="#1f77b4"
                ),
                side="left"
            ),
            yaxis2=dict(
                title="Deaths",
                titlefont=dict(
                    color="#ff7f0e"
                ),
                tickfont=dict(
                    color="#ff7f0e"
                ),
                anchor="free",
                overlaying="y",
                side="right",
                position=0.95
            )
        )
        
        fig.update_layout(
        title_text=self.name,
        width=800,
        )
        
        fig.show()


In [4]:
pa = State('Pennsylvania',50,7)
pa.create()

In [5]:
fl = State('Florida',50,7)
fl.create()

In [6]:
il = State('Illinois',50,7)
il.create()

In [7]:
mn = State('Minnesota',50,7)
mn.create()

In [8]:
ny = State('New York',50,7)
ny.create()

In [9]:
tx = State('Texas',50,7)
tx.create()

In [10]:
ca = State("California",50,7)
ca.create()

In [11]:
az = State("Arizona",50,7)
az.create()

In [12]:
mydict = {"PA" : pa,
          "FL" : fl,
          "IL" : il,
          "NY" : ny,
          "MN" : mn,
          "TX" : tx,
          "CA" : ca,
          "AZ" : az}
     

# Assign an empty figure widget with two traces
trace1 = go.Scatter(x=pa.dates, y=pa.confirmed, opacity=0.75, name='Confirmed')
trace2 = go.Scatter(x=pa.dates, y=pa.deaths   , opacity=0.75, name='Deaths')
fig = go.FigureWidget(#data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text=pa.name
                        )
                    ))

fig.add_trace(go.Scatter(
    x=pa.dates,      
    y=pa.confirmed,
    name="Confirmed"
))

fig.add_trace(go.Scatter(

    x=pa.dates,    
    y=pa.deaths,
    name="Deaths",
    yaxis="y2"
))

fig.update_layout(
    title_text=pa.name,
    width=900,
    xaxis=dict(
        domain=[0.06, 0.94]
    ),
    yaxis=dict(
        title="Confirmed",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        ),
        side="left"
    ),
    yaxis2=dict(
        title="Deaths",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.95
    )
)


states = widgets.Dropdown(
    options = sorted(list(mydict.keys())), 
    value = 'PA',
    description='States:',
)


def response(v):
    x  = mydict[states.value].dates
    y1 = mydict[states.value].confirmed
    y2 = mydict[states.value].deaths
    ti = mydict[states.value].name

    with fig.batch_update():
        fig.data[0].y = y1
        fig.data[1].y = y2

    fig.update_layout(
         title_text=ti,
     )

states.observe(response,names="value")   

container = widgets.HBox(children=[states])
widgets.VBox([container,
              fig   ])

In [13]:
## These might be needed before you run this notebook:

#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!pip install plotly